# IMPORTS

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from collections import Counter

from df_functions import count_aromaticAA
from profile_distances import make_profile
import string
import seaborn as sns
import joblib
import matplotlib
import numpy as np
from IPython import display
from pyteomics import electrochem, mass, parser
from sklearn import feature_extraction, metrics, preprocessing, model_selection

from sklearn import svm, metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

Data is from 'receptor_full_v3(14 MARCH).csv'. It is the original csv downloaded from IEDB, filtered for 'B cell response' and 'Homo sapiens' as the Heavy Chain species. The humanbcell.csv is exported from the DBcompare notebook under OTHER.

# FEATURES

In [2]:
!ls

BCRTESTDFfeatures.csv
BCRdataCDRpept_nonpept.csv
BCRdataCDRpeptnonpept_LPS.csv
BCRdataCDRpeptnonpept_LPS_enriched.csv
HUMANBCRdataCDRpept_nonpept.csv
HUMANBCRdataCDRpeptnonpept_LPS_enriched.csv
KABABCRdataCDRpeptnonpept_LPS_enrichedfeatures.csv
KABATBCRdataCDRpeptnonpept_LPS_enriched.csv
KABATBCRdataCDRpeptnonpept_PNEUMO.csv
RFCDR3peptnonpept_LPS_enriched.joblib
TESTBCRdataCDRpeptnonpept_LPS_enriched.csv
__pycache__
data
data_labeling.ipynb
df_functions.py
epitope_labels.pdf
featurematrix.ipynb
featurespept_nonpept.csv
featurespeptnonpept_LPS.csv
featurespeptnonpept_LPS_enriched.csv
figdir
get_data_IEDB.ipynb
nonLPS_peptide.csv
nonLPSfeautrues.csv
non_peptide_clf.ipynb
non_peptides.csv
organism_dict.json
pneumotestje.csv
profile_distances.py


In [3]:
datadir = Path.cwd().parents[0] / 'DATA_FILES' 

In [4]:
#data (see above)
#file = 'BCRdataCDRpept_nonpept.csv'
#file = 'BCRdataCDRpeptnonpept_LPS.csv'
#file = 'BCRdataCDRpeptnonpept_LPS_enriched.csv'
#file = 'TESTBCRdataCDRpeptnonpept_LPS_enriched.csv'
#file = 'KABATBCRdataCDRpeptnonpept_LPS_enriched.csv'
file='pneumotestje.csv'



data = pd.read_csv(file, dtype='object', sep = ',')
regions = data['sequence'].unique()
print(data.shape)
print(list(data))
#Weird notations which couldn't pass the feature generation pipeline and raised an error (KeyError)
data.drop(data.index[data.sequence == 'QSLLDSOGKTY'], inplace=True) #Pyrrolysine: 'O' amino acid
data.drop(data.index[data.sequence == 'lYIGNGYS'], inplace=True) #Bad notation
data.drop(data.index[data.sequence == 'SK QQVSGGFYWYFDL'], inplace=True) #space included
data.drop(data.index[data.sequence == 'LLDSOGKTY'], inplace=True) #Pyrrolysine: 'O' amino acid
data.drop(data.index[data.sequence == 'IGNGYS'], inplace=True) #Bad notation
data.drop(data.index[data.sequence == ' QQVSGGFYWYFDL'], inplace=True) #space included
data.info()

(4841, 3)
['sequence', 'region', 'peptide_label']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4839 entries, 0 to 4840
Data columns (total 3 columns):
sequence         4839 non-null object
region           4839 non-null object
peptide_label    4839 non-null object
dtypes: object(3)
memory usage: 151.2+ KB


In [5]:
data

,sequence,region,peptide_label
0,LGDYGYAYTMDY,HCDR3,0
1,GLDS,HCDR3,0
2,YYDDHYCLDY,HCDR3,0
3,YPYYYGTSHWYFDV,HCDR3,0
4,RAPFYGNHAMDY,HCDR3,0
5,MWYYGTYYFDY,HCDR3,1
6,MWYYGTYYFDY,HCDR3,0
7,SHYGLDWNFDV,HCDR3,0
8,WSFLY,HCDR3,0
9,NYYGMDY,HCDR3,0


In [6]:
list_test_HCDR3s_1 = ['CARDWYW','CVRDIYSFGSRDGMDWYW','CARDHDGYYERFSWYW','CARDINPGSDGYYDALDWYW','CARDMRRFFDDGDAMDWYW']
list_test_HCDR3s_0 = ['CARDYYGSGNYYNPKPLDVW','CARESYYELWTGTYPGWELDYW','CASLYDSRGYYWVFDYW','CARNPPLYNFRSDSQVFHYMDVW','CARGAWFDFW', 'CARGGVADVW']
region_test = ['HCDR'] 
HCDR3_test = list_test_HCDR3s_1+list_test_HCDR3s_0
print(len(HCDR3_test))
region_test = ['HCDR3']*11
list_test_labels = [1,1,1,1,1,0,0,0,0,0,0]

TEST_DF = pd.DataFrame({'sequence': HCDR3_test, 'region': region_test, 'peptide_label':list_test_labels})

11


In [7]:
mutation_stability = {'A': 13, 'C': 52, 'D': 11, 'E': 12, 'F': 32, 'G': 27, 'H': 15, 'I': 10,
                      'K': 24, 'L': 34, 'M':  6, 'N':  6, 'P': 20, 'Q': 10, 'R': 17, 'S': 10,
                      'T': 11, 'V': 17, 'W': 55, 'Y': 31, 'X': 21}

basicity = {'A': 206.4, 'B': 210.7, 'C': 206.2, 'D': 208.6, 'E': 215.6, 'F': 212.1, 'G': 202.7,
            'H': 223.7, 'I': 210.8, 'K': 221.8, 'L': 209.6, 'M': 213.3, 'N': 212.8, 'P': 214.4,
            'Q': 214.2, 'R': 237.0, 'S': 207.6, 'T': 211.7, 'V': 208.7, 'W': 216.1, 'X': 210.2,
            'Y': 213.1, 'Z': 214.9}

hydrophobicity = {'A': 0.16, 'B': -3.14, 'C': 2.50, 'D': -2.49, 'E': -1.50, 'F': 5.00, 'G': -3.31,
                  'H': -4.63, 'I': 4.41, 'K': -5.00, 'L': 4.76, 'M': 3.23, 'N': -3.79, 'P': -4.92,
                  'Q': -2.76, 'R': -2.77, 'S': -2.85, 'T': -1.08, 'V': 3.02, 'W': 4.88, 'X': 4.59,
                  'Y': 2.00, 'Z': -2.13}

helicity = {'A': 1.24, 'B': 0.92, 'C': 0.79, 'D': 0.89, 'E': 0.85, 'F': 1.26, 'G': 1.15, 'H': 0.97,
            'I': 1.29, 'K': 0.88, 'L': 1.28, 'M': 1.22, 'N': 0.94, 'P': 0.57, 'Q': 0.96, 'R': 0.95,
            'S': 1.00, 'T': 1.09, 'V': 1.27, 'W': 1.07, 'X': 1.29, 'Y': 1.11, 'Z': 0.91}

aromatic_AA = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 1, 'G': 0, 'H': 1,
            'I': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'P': 0, 'Q': 0, 'R': 0,
            'S': 0, 'T': 0, 'V': 0, 'W': 1, 'X': 0, 'Y': 1, 'Z':0}

In [8]:
features_list = []

In [9]:
#sequence length
features_list.append(data['sequence'].apply(
        lambda sequence: parser.length(sequence)).to_frame()
                     .rename(columns={'sequence': 'sequence_length'}))

In [10]:
# number of occurences of each amino acid
aa_counts = pd.DataFrame.from_records(
    [parser.amino_acid_composition(sequence) for sequence in data['sequence']]).fillna(0)
aa_counts.columns = ['count_{}'.format(column) for column in aa_counts.columns]
features_list.append(aa_counts)

In [11]:
#physicochemical properties: (average) basicity, (average) hydrophobicity,
#                            (average) helicity, pI, (average) mutation stability,
#                            (average) molecular weight, (average) AAA count

features_list.append(data['sequence'].apply(
        lambda seq: sum(make_profile(seq, prop='basicity')) / parser.length(seq))
                     .to_frame().rename(columns={'sequence': 'avg_basicity'}))
features_list.append(data['sequence'].apply(
        lambda seq: sum(make_profile(seq, prop='hydrophobicity')) / parser.length(seq))
                     .to_frame().rename(columns={'sequence': 'avg_hydrophobicity'}))
features_list.append(data['sequence'].apply(
        lambda seq: sum(make_profile(seq, prop='helicity')) / parser.length(seq))
                     .to_frame().rename(columns={'sequence': 'avg_helicity'}))
features_list.append(data['sequence'].apply(
        lambda seq: electrochem.pI(seq)).to_frame().rename(columns={'sequence': 'pI'}))
features_list.append(data['sequence'].apply(
        lambda seq: sum(make_profile(seq, prop='mutation_stability')) / parser.length(seq))
                     .to_frame().rename(columns={'sequence': 'avg_mutation_stability'}))
features_list.append(data['sequence'].apply(
        lambda seq: sum(make_profile(seq, prop='molecular_weight')) / parser.length(seq))
                     .to_frame().rename(columns={'sequence': 'avg_molecular_weight'}))
#features_list.append(data[sequence].apply(
#        lambda seq: sum(count_aromaticAA(seq)) / parser.length(seq))
#                     .to_frame().rename(columns={sequence: 'avg_AAA'}))

In [12]:
pos_aa, pos_basicity, pos_hydro, pos_helicity, pos_pI, pos_mutation, pos_aro = [[] for _ in range(7)]
for sequence in data['sequence']:
    length = parser.length(sequence)
    start_pos = -1 * (length // 2)
    pos_range = list(range(start_pos, start_pos + length)) if length % 2 == 1 else\
                list(range(start_pos, 0)) + list(range(1, start_pos + length + 1))
    
    pos_aa.append({'pos_{}_{}'.format(pos, aa): 1
                   for pos, aa in zip(pos_range, sequence)})
    pos_basicity.append({'pos_{}_basicity'.format(pos): basicity[aa]
                         for pos, aa in zip(pos_range, sequence)})
    pos_hydro.append({'pos_{}_hydrophobicity'.format(pos): hydrophobicity[aa]
                      for pos, aa in zip(pos_range, sequence)})
    pos_helicity.append({'pos_{}_helicity'.format(pos): helicity[aa]
                         for pos, aa in zip(pos_range, sequence)})
    pos_pI.append({'pos_{}_pI'.format(pos): electrochem.pI(aa)
                   for pos, aa in zip(pos_range, sequence)})
    pos_mutation.append({'pos_{}_mutation_stability'.format(pos): mutation_stability[aa]
                         for pos, aa in zip(pos_range, sequence)})
    #pos_aro.append({'pos_{}_aromatic'.format(pos): aromatic_AA[aa]
                         #for pos, aa in zip(pos_range, sequence)})

features_list.append(pd.DataFrame.from_records(pos_aa).fillna(0))
features_list.append(pd.DataFrame.from_records(pos_basicity).fillna(0))
features_list.append(pd.DataFrame.from_records(pos_hydro).fillna(0))
features_list.append(pd.DataFrame.from_records(pos_helicity).fillna(0))
features_list.append(pd.DataFrame.from_records(pos_pI).fillna(0))
features_list.append(pd.DataFrame.from_records(pos_mutation).fillna(0))
features_list.append(pd.DataFrame.from_records(pos_aro).fillna(0))

In [13]:
# combine all features
BCRfeatures = pd.concat(features_list, axis=1)
print('Samples: {} - features: {}'.format(BCRfeatures.shape[0], BCRfeatures.shape[1]))

Samples: 4841 - features: 649


In [14]:
BCRfeatures['peptide_label'] = data['peptide_label']
BCRfeatures['region'] = data['region']

In [15]:
len(BCRfeatures.columns)

651

In [16]:
#Strange things happen with NaN values appearing
null_data = BCRfeatures[BCRfeatures.isnull().any(axis=1)]
print('Null data:',len(null_data))

BCRfeatures.drop([col for col in BCRfeatures.columns if 'X' in col], axis=1, inplace=True)

print(BCRfeatures.shape)
BCRfeatures.dropna(inplace=True)
print(BCRfeatures.shape)

#BCRfeatures.to_csv('featurespept_nonpept.csv', index=False)
#BCRfeatures.to_csv('featurespeptnonpept_LPS.csv', index=False)
#BCRfeatures.to_csv('featurespeptnonpept_LPS_enriched.csv', index=False)
#BCRfeatures.to_csv('BCRTESTDFfeatures.csv', index=False)
BCRfeatures.to_csv('pneumoootjes.csv', index=False)

Null data: 4
(4841, 644)
(4837, 644)
